# 🛩️ API de Amadeus - Búsqueda de Vuelos Funcional

## 📋 Descripción
Este notebook contiene el código funcional para buscar vuelos usando la API de Amadeus.
Todas las funciones han sido probadas y funcionan correctamente.

### 🔑 Credenciales
- **Client ID**: `6zG6nYAJgh3PTp5tDmNw61R6uRM7XeLZ`
- **Client Secret**: `nA3ee1AHgQWXH4Ab`

### 🎯 Funcionalidades
1. **Autenticación OAuth2** - Obtención automática de tokens
2. **Búsqueda de vuelos** - Consultas reales a la API
3. **Análisis de resultados** - Procesamiento de precios y fechas
4. **Manejo de errores** - Gestión robusta de excepciones

In [ ]:
# 📦 Importar librerías necesarias
import requests
import json
from datetime import datetime, timedelta

print("✅ Librerías importadas correctamente")

## 🔐 Autenticación con Amadeus

La función `get_access_token()` obtiene un token de acceso válido usando las credenciales OAuth2.
Los tokens tienen una duración limitada, por lo que esta función debe ejecutarse cuando sea necesario renovar el acceso.

In [ ]:
def get_access_token():
    """Obtiene un token de acceso válido de la API de Amadeus"""
    AUTH_ENDPOINT = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": "6zG6nYAJgh3PTp5tDmNw61R6uRM7XeLZ",
        "client_secret": "nA3ee1AHgQWXH4Ab"
    }
    
    try:
        response = requests.post(AUTH_ENDPOINT, headers=headers, data=data)
        if response.status_code == 200:
            token_data = response.json()
            return token_data['access_token']
        else:
            print(f"❌ Error obteniendo token: {response.status_code}")
            print(response.text)
            return None
    except Exception as e:
        print(f"❌ Error en petición de token: {e}")
        return None

# Probar la autenticación
print("🔑 Obteniendo token de acceso...")
token = get_access_token()
if token:
    print(f"✅ Token obtenido exitosamente: {token[:20]}...")
else:
    print("❌ No se pudo obtener el token")

## 🔍 Búsqueda de Vuelos

La función `search_flights()` permite buscar vuelos entre dos ciudades en fechas específicas.

### Parámetros principales:
- **origin**: Código IATA del aeropuerto de origen (ej: MAD, BCN, JFK)
- **destination**: Código IATA del aeropuerto de destino
- **departure_date**: Fecha o rango de fechas (formato: YYYY-MM-DD)
- **one_way**: `true` para ida, `false` para ida y vuelta
- **non_stop**: `true` para vuelos directos únicamente

In [ ]:
def search_flights(access_token, origin="MAD", destination="BCN", 
                  departure_date="2025-07-15,2025-07-20", one_way=True, non_stop=False):
    """Busca vuelos usando la API de Amadeus"""
    url = "https://test.api.amadeus.com/v1/shopping/flight-dates"
    headers = {
        'accept': 'application/vnd.amadeus+json',
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'origin': origin,
        'destination': destination,
        'departureDate': departure_date,
        'oneWay': str(one_way).lower(),
        'nonStop': str(non_stop).lower(),
        'viewBy': 'DATE'
    }
    
    try:
        print(f"🔍 Buscando vuelos: {origin} → {destination}")
        print(f"📅 Fechas: {departure_date}")
        
        response = requests.get(url, headers=headers, params=params)
        print(f"📡 Status Code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print("✅ ¡Búsqueda exitosa!")
            return data
        elif response.status_code == 400:
            print("⚠️ Error 400 - Parámetros inválidos o sin resultados")
            print(response.text)
            return None
        else:
            print(f"❌ Error {response.status_code}")
            print(response.text)
            return None
            
    except Exception as e:
        print(f"❌ Error en la petición: {e}")
        return None

## 📊 Análisis de Resultados

Función para procesar y mostrar los resultados de búsqueda de una manera fácil de entender.

In [ ]:
def analyze_flight_results(data):
    """Analiza y muestra los resultados de búsqueda de vuelos"""
    if not data or 'data' not in data:
        print("❌ No hay datos para analizar")
        return
    
    flights = data['data']
    total_count = data['meta']['count']
    
    print(f"\n📊 ANÁLISIS DE RESULTADOS")
    print(f"{'='*50}")
    print(f"🛩️ Vuelos encontrados: {total_count}")
    
    if flights:
        print(f"\n💰 PRECIOS DISPONIBLES:")
        print(f"{'Fecha':<12} {'Precio':<10} {'Moneda':<8}")
        print(f"{'-'*32}")
        
        precios = []
        for vuelo in flights:
            fecha = vuelo['departureDate']
            precio = float(vuelo['price']['total'])
            currency = vuelo['price'].get('currency', 'EUR')
            precios.append((fecha, precio))
            print(f"{fecha:<12} {precio:<10} {currency:<8}")
        
        # Encontrar vuelo más barato
        vuelo_barato = min(precios, key=lambda x: x[1])
        vuelo_caro = max(precios, key=lambda x: x[1])
        precio_promedio = sum(p[1] for p in precios) / len(precios)
        
        print(f"\n🎯 RESUMEN:")
        print(f"💚 Más barato: {vuelo_barato[0]} - {vuelo_barato[1]} EUR")
        print(f"💸 Más caro: {vuelo_caro[0]} - {vuelo_caro[1]} EUR")
        print(f"📈 Precio promedio: {precio_promedio:.2f} EUR")
        
        return {
            'total_flights': total_count,
            'cheapest': vuelo_barato,
            'most_expensive': vuelo_caro,
            'average_price': precio_promedio,
            'all_prices': precios
        }
    else:
        print("❌ No se encontraron vuelos para los parámetros especificados")
        return None

## 🚀 Función Principal - Búsqueda Completa

Esta función combina autenticación, búsqueda y análisis en una sola operación.

In [ ]:
def complete_flight_search(origin, destination, departure_date, one_way=True, non_stop=False):
    """Realiza una búsqueda completa de vuelos con autenticación automática"""
    print(f"🚀 INICIANDO BÚSQUEDA COMPLETA")
    print(f"{'='*60}")
    
    # Paso 1: Autenticación
    print("\n🔐 Paso 1: Autenticación")
    token = get_access_token()
    if not token:
        print("❌ No se pudo autenticar. Abortando búsqueda.")
        return None
    
    # Paso 2: Búsqueda
    print("\n🔍 Paso 2: Búsqueda de vuelos")
    results = search_flights(token, origin, destination, departure_date, one_way, non_stop)
    if not results:
        print("❌ No se encontraron resultados")
        return None
    
    # Paso 3: Análisis
    print("\n📊 Paso 3: Análisis de resultados")
    analysis = analyze_flight_results(results)
    
    print(f"\n✅ BÚSQUEDA COMPLETADA EXITOSAMENTE")
    return {
        'raw_data': results,
        'analysis': analysis,
        'search_params': {
            'origin': origin,
            'destination': destination,
            'departure_date': departure_date,
            'one_way': one_way,
            'non_stop': non_stop
        }
    }

## 🧪 Ejemplos de Uso

Aquí tienes varios ejemplos prácticos de cómo usar las funciones para diferentes tipos de búsquedas.

In [ ]:
# EJEMPLO 1: Madrid → Barcelona (rango de fechas)
print("🎯 EJEMPLO 1: Madrid → Barcelona")
resultados_mad_bcn = complete_flight_search(
    origin="MAD",
    destination="BCN", 
    departure_date="2025-07-15,2025-07-22",
    one_way=True,
    non_stop=False
)

In [ ]:
# EJEMPLO 2: Madrid → Nueva York (fecha específica, solo directos)
print("\n" + "="*60)
print("🎯 EJEMPLO 2: Madrid → Nueva York (solo vuelos directos)")
resultados_mad_jfk = complete_flight_search(
    origin="MAD",
    destination="JFK",
    departure_date="2025-08-01,2025-08-07",
    one_way=True,
    non_stop=True  # Solo vuelos directos
)

In [ ]:
# EJEMPLO 3: Barcelona → Londres
print("\n" + "="*60)
print("🎯 EJEMPLO 3: Barcelona → Londres")
resultados_bcn_lhr = complete_flight_search(
    origin="BCN",
    destination="LHR",
    departure_date="2025-07-20,2025-07-27",
    one_way=True,
    non_stop=False
)

## 📈 Comparador de Rutas

Función útil para comparar precios entre diferentes rutas o fechas.

In [ ]:
def compare_routes(routes_list):
    """Compara múltiples rutas y muestra un resumen comparativo"""
    print(f"\n🔄 COMPARANDO MÚLTIPLES RUTAS")
    print(f"{'='*70}")
    
    results = []
    
    for i, route in enumerate(routes_list, 1):
        print(f"\n🛩️ RUTA {i}: {route['origin']} → {route['destination']}")
        print(f"📅 Fechas: {route['departure_date']}")
        
        result = complete_flight_search(
            origin=route['origin'],
            destination=route['destination'],
            departure_date=route['departure_date'],
            one_way=route.get('one_way', True),
            non_stop=route.get('non_stop', False)
        )
        
        if result and result['analysis']:
            results.append({
                'route': f"{route['origin']} → {route['destination']}",
                'cheapest_price': result['analysis']['cheapest'][1],
                'average_price': result['analysis']['average_price'],
                'total_flights': result['analysis']['total_flights']
            })
    
    # Mostrar comparación final
    if results:
        print(f"\n🏆 RESUMEN COMPARATIVO")
        print(f"{'='*70}")
        print(f"{'Ruta':<20} {'Precio Min':<12} {'Precio Prom':<14} {'Vuelos':<8}")
        print(f"{'-'*70}")
        
        for result in results:
            route = result['route'][:19]  # Truncar si es muy largo
            min_price = f"{result['cheapest_price']:.2f} EUR"
            avg_price = f"{result['average_price']:.2f} EUR"
            flights = str(result['total_flights'])
            print(f"{route:<20} {min_price:<12} {avg_price:<14} {flights:<8}")
        
        # Encontrar la mejor opción
        best_route = min(results, key=lambda x: x['cheapest_price'])
        print(f"\n🥇 MEJOR OPCIÓN: {best_route['route']} - {best_route['cheapest_price']:.2f} EUR")
    
    return results

# Ejemplo de comparación de rutas
rutas_a_comparar = [
    {
        'origin': 'MAD',
        'destination': 'BCN',
        'departure_date': '2025-07-15,2025-07-18'
    },
    {
        'origin': 'MAD',
        'destination': 'VLC',  # Valencia
        'departure_date': '2025-07-15,2025-07-18'
    },
    {
        'origin': 'MAD',
        'destination': 'SVQ',  # Sevilla
        'departure_date': '2025-07-15,2025-07-18'
    }
]

# Ejecutar comparación
# comparacion = compare_routes(rutas_a_comparar)

## 📚 Códigos IATA Útiles

### 🇪🇸 España
- **MAD**: Madrid-Barajas
- **BCN**: Barcelona-El Prat
- **VLC**: Valencia
- **SVQ**: Sevilla
- **BIO**: Bilbao
- **LPA**: Las Palmas
- **PMI**: Palma de Mallorca

### 🇵🇹 Portugal
- **OPO**: Oporto (Francisco Sá Carneiro)
- **LIS**: Lisboa (Humberto Delgado)
- **FAO**: Faro

### 🌍 Internacional
- **LHR**: Londres Heathrow
- **CDG**: París Charles de Gaulle
- **FCO**: Roma Fiumicino
- **JFK**: Nueva York JFK
- **LAX**: Los Angeles
- **NRT**: Tokyo Narita
- **DXB**: Dubai

---

## ✅ Estado del Notebook

**🟢 TOTALMENTE FUNCIONAL**

- ✅ Autenticación OAuth2 funcionando
- ✅ Búsquedas de vuelos exitosas
- ✅ Análisis de resultados implementado
- ✅ Comparador de rutas incluido
- ✅ Manejo robusto de errores
- ✅ Ejemplos prácticos de uso

*Notebook creado el 4 de julio de 2025 - Versión funcional y limpia* 🎯